# Training Routine

In [9]:
from datasets import load_dataset, DatasetDict

## Load Data

In [8]:
full_dataset = load_dataset('parquet', data_files={'train': 'data/agent_compt_cleaned.parquet'})

In [18]:
train_split = full_dataset['train'].train_test_split(test_size=0.2)
val_split = train_split['test'].train_test_split(test_size=0.5)

In [23]:
dataset = DatasetDict({
    'train': train_split['train'],
    'test': val_split['test'],
    'validation': val_split['train']
})

In [24]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 68974
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 8622
    })
    validation: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 8622
    })
})